# Unsupervised clustering on rock properties

Sometimes we don't have labels, but would like to discover structure in a dataset. This is what clustering algorithms attempt to do. They don't require labels from us &mdash; they are 'unsupervised'.

We'll use a subset of the [Rock Property Catalog](http://subsurfwiki.org/wiki/Rock_Property_Catalog) data, licensed CC-BY Agile Scientific. Note that the data have been preprocessed, including the addition of noise. See the notebook [RPC_for_regression_and_classification.ipynb](RPC_for_regression_and_classification.ipynb). 

We'll use two unsupervised techniques:

- k-means clustering
- DBSCAN

We do have lithology labels for this dataset, so we can use those as a measure of how well we're doing with the clustering.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/RPC_4_lithologies.csv')

In [ ]:
df.describe()

Notice that the count of `Rho` values is smaller than for the other properties.

Pairplots are a good way to see how the various features are distributed with respect to each other:

In [ ]:
sns.pairplot(df.dropna(), vars=['Vp', 'Vs', 'Rho_n'], hue='Lithology')

## Clustering with _k_-means

From [the Wikipedia article](https://en.wikipedia.org/wiki/K-means_clustering):

> k-means clustering is a method of vector quantization, originally from signal processing, that is popular for cluster analysis in data mining. k-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster. This results in a partitioning of the data space into Voronoi cells.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
clu = KMeans()

cols = ['Vp', 'Vs', 'Rho_n']

clu.fit(df[cols].values)

The old classic: NaNs. Remember the count of `Rho` points being smaller than the others?

The easiest thing to do, assuming we have the data, is to drop the rows with NaNs.

In [ ]:
df = df.dropna()

In [ ]:
clu.fit(df[cols].values)

In [ ]:
df['K means'] = clu.predict(df[cols].values)

In [ ]:
for name, group in df.groupby('K means'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

We actually do have the labels, so let's compare...

In [ ]:
for name, group in df.groupby('Lithology'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

## Clustering with DBSCAN

DBSCAN has nothing to do with databases. From [the Wikipedia article](https://en.wikipedia.org/wiki/DBSCAN):

> Density-based spatial clustering of applications with noise (DBSCAN) is [...] a density-based clustering algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature.

In [ ]:
from sklearn.cluster import DBSCAN

DBSCAN()

There are two important hyperparameters:

- `eps`, the maximum distance between points in the same cluster.
- `min_samples`, the minimum number of samples in a cluster.

In [ ]:
clu = DBSCAN(eps=150, min_samples=10)

clu.fit(df[cols].values)

In [ ]:
df['DBSCAN'] = clu.labels_

In [ ]:
for name, group in df.groupby('DBSCAN'):
    plt.scatter(group.Vp, group.Rho_n, label=name)

It's a bit hard to juggle the two parameters... let's make an interactive widget:

In [ ]:
from ipywidgets import interact

@interact(a=10, b=10)
def add(a, b):
    return a + b

In [ ]:
@interact(eps=(10, 250, 10))
def plot(eps):
    clu = DBSCAN(eps=eps)
    clu.fit(df[cols].values)
    df['DBSCAN'] = clu.labels_
    for name, group in df.groupby('DBSCAN'):
        plt.scatter(group.Vp, group.Rho_n, label=name)

## Manifold embedding with t-SNE

[`t-SNE`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) was recently implemented in `sklearn`. t-statistic neighbourhood embedding is a popular and very effective dimensionality reduction strategy. The caveat is that distance between clusters is not typically meaningful. But it's at least a useful data exploration tool.

Usually we want `n_components=2`, so we get the data into a 2-space, e.g. for cross-plotting.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(init='random', perplexity=15)

embedding = tsne.fit_transform(df[cols].values)

In [ ]:
from sklearn.preprocessing import LabelEncoder

labels = LabelEncoder().fit_transform(df.Lithology)

In [ ]:
plt.scatter(*embedding.T, c=labels, cmap='tab10')

Another popular tool is UMAP. Check out [`umap-learn`](https://pypi.org/project/umap-learn/). You can install it with 

    conda install -y -c conda-forge umap-learn
    
It has the same interface as `sklearn` so it's very easy to use.

## Taking it further

There are metrics for comparing clusterings. For example, `adjusted_rand_score` &mdash; from the scikit-learn docs:

> The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
>
> The raw RI score is then “adjusted for chance” into the ARI score using the following scheme:
> 
> ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)
> 
> The adjusted Rand index is thus ensured to have a value close to 0.0 for random labeling independently of the number of clusters and samples and exactly 1.0 when the clusterings are identical (up to a permutation).

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(df.Lithology, df.DBSCAN)

<div class="alert alert-success">
<h3>Exercises</h3>

- Can you make the interactive widget display the Rand score? Use `plt.text(x, y, "Text")`.
- Can you write a loop to find the value of `eps` giving the highest Rand score?
- Can you add the `min_samples` parameter to the widget?
- Explore some of [the other clustering algorithms](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).
- Try some clustering on one of your own datasets (or use something from [sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets), e.g. `sklearn.datasets.load_iris`).
</div>